# 0. Import libraries

The libraries used to create the data are in the following cell

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

2023-09-22 11:42:10.547304: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 11:42:11.094587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1. Define constants

This section defines constants to be used in the notebook

In [2]:
# Path constants
DATA_FOLDER_PATH = "./Data"
FEATURES_FILENAME = "surfaces.npy"
LABELS_FILENAME = "coefficients.npy"

FEATURES_FILE_PATH = f"{DATA_FOLDER_PATH}/{FEATURES_FILENAME}"
LABELS_FILE_PATH = f"{DATA_FOLDER_PATH}/{LABELS_FILENAME}"

# Column names
SURFACE_POINTS = "surface_points"
ZERNIKE_COEFFICIENTS = "zernike_coefficients"

# Neural network architecture size
INPUT_SIZE = 60
OUTPUT_SIZE = 21

# Neural network training parameters
N_EPOCHS = 100
BATCH_SIZE = 512
LEARNING_RATE = 0.01
ACTIVATION = 'relu'
N_HIDDEN = [512, 64]
REGULARIZER = keras.regularizers.L1L2(l1=0.001,l2=0.1)
INITIALIZER = keras.initializers.he_normal(seed=None)
LOSS = tf.keras.losses.MeanSquaredError()
OPTIMIZER = Adam(learning_rate=LEARNING_RATE, beta_1=0.9, beta_2=0.999)
METRICS = tf.keras.metrics.MeanSquaredError()

2023-09-22 11:42:12.777563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 11:42:12.797542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 11:42:12.797735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# 2. Compile functions

In [3]:
def get_features_and_labels_from_df(df):
    """
    Converts the columns of a surface dataframe into a list of features and labels in the form of list of numpy arrays
    Input:
        df (pd.DataFrame): The surface dataframe with columns 'surface_points'(features) and 'zernike_coefficients'(labels)
    
    Output:
        features (np.array): A numpy array with the z values of the surface at sampled points
        labels (np.array): A numpy array with the Zernike coefficients of the surface
    """
    features = convert_column_into_numpy_array_list(df, SURFACE_POINTS)
    labels = convert_column_into_numpy_array_list(df, ZERNIKE_COEFFICIENTS)
    return features, labels
    

def convert_column_into_numpy_array_list(df, column_name):
    """
    Converts the specified column of the dataframe into a list of numpy arrays
    Input:
        df (pd.DataFrame): The surface dataframe with columns 'surface_points' and 'zernike_coefficients'
        column_name (string): The name of the column to convert to list of numpy arrays
        
    Returns:
        numpy_list (tensor): The list of numpy arrays converted from the column
    """
    numpy_series = df[column_name].apply(lambda x: np.fromstring(x[1: -1], dtype=float, sep=' ')).to_list()
    numpy_list = tf.data.Dataset.from_tensor_slices(numpy_series)
    return numpy_series


def create_architecture():
    model = keras.Sequential(name="SurfaceReconstructor")
    model.add(keras.layers.InputLayer(input_shape=(INPUT_SIZE,),
                                      batch_size=None))
    
    for neurons in N_HIDDEN:
        model.add(keras.layers.Dense(neurons,
                                     kernel_regularizer=REGULARIZER,
                                     kernel_initializer=keras.initializers.HeNormal(seed=None),
                                     use_bias=False
                                     ))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Activation(ACTIVATION))
        
    model.add(keras.layers.Dense(OUTPUT_SIZE,
                                 activation="linear"
                                ))
    
    model_name = "SurfaceReconstructor"
    return model, model_name


def compile_model(my_model):
    my_model.compile(loss=LOSS,
                     optimizer=OPTIMIZER,
                     metrics=[METRICS])
    

# 2. Read data

First read the data

In [4]:
features = np.load(FEATURES_FILE_PATH, allow_pickle=True)
labels = np.load(LABELS_FILE_PATH, allow_pickle=True)

Split the data:
- **80%** training
- **10%** validation/dev test
- **10%** final test

In [5]:
# Calculate the number of rows for each split
total_rows = len(features)
train_size = int(0.8 * total_rows)
val_size = int(0.1 * total_rows)

# Split the DataFrame into training (80%), validation (10%), and test (10%)
train_features = features[:train_size]
train_labels = labels[:train_size]

val_features = features[train_size:train_size + val_size]
val_labels = labels[train_size:train_size + val_size]

test_features = features[train_size + val_size:]
test_labels = labels[train_size + val_size:]

# 3. Train neural network

Create the neural network architecture and compile it

In [6]:
surface_reconstruction_model, model_name = create_architecture()
compile_model(surface_reconstruction_model)

In [7]:
history = surface_reconstruction_model.fit(train_features,
                                           train_labels,
                                           batch_size=BATCH_SIZE,
                                           epochs=N_EPOCHS,
                                           validation_data= (val_features, val_labels),
                                           verbose=1)

Epoch 1/100


2023-09-22 11:42:14.778132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-22 11:42:14.781229: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1cbebbf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-22 11:42:14.781258: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-09-22 11:42:14.785621: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-22 11:42:14.923335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-22 11:42:15.022948: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime o

47/47 [==============================] - 3s 8ms/step - loss: 38.6772 - mean_squared_error: 32.8540 - val_loss: 33.3402 - val_mean_squared_error: 33.1726
Epoch 2/100
47/47 [==============================] - 0s 5ms/step - loss: 32.7584 - mean_squared_error: 32.6694 - val_loss: 33.1755 - val_mean_squared_error: 33.1142
Epoch 3/100
47/47 [==============================] - 0s 4ms/step - loss: 32.7177 - mean_squared_error: 32.6634 - val_loss: 33.1214 - val_mean_squared_error: 33.0750
Epoch 4/100
47/47 [==============================] - 0s 5ms/step - loss: 32.7089 - mean_squared_error: 32.6628 - val_loss: 32.9696 - val_mean_squared_error: 32.9230
Epoch 5/100
47/47 [==============================] - 0s 5ms/step - loss: 32.7032 - mean_squared_error: 32.6605 - val_loss: 32.8847 - val_mean_squared_error: 32.8456
Epoch 6/100
47/47 [==============================] - 0s 5ms/step - loss: 32.6923 - mean_squared_error: 32.6575 - val_loss: 32.8238 - val_mean_squared_error: 32.7896
Epoch 7/100
47/47 [===

47/47 [==============================] - 0s 5ms/step - loss: 32.6861 - mean_squared_error: 32.6567 - val_loss: 32.6574 - val_mean_squared_error: 32.6273
Epoch 51/100
47/47 [==============================] - 0s 5ms/step - loss: 32.6871 - mean_squared_error: 32.6577 - val_loss: 32.6596 - val_mean_squared_error: 32.6288
Epoch 52/100
47/47 [==============================] - 0s 6ms/step - loss: 32.6882 - mean_squared_error: 32.6578 - val_loss: 32.6535 - val_mean_squared_error: 32.6238
Epoch 53/100
 1/47 [..............................] - ETA: 0s - loss: 32.6886 - mean_squared_error: 32.6589

KeyboardInterrupt: 

In [ ]:
results=pd.DataFrame(history.history)
results.plot(figsize=(8, 5))
plt.grid(True)
plt.xlabel ("Epochs")
plt.ylabel ("Mean Squared Error")
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()